# 1. Setup

## 1.1 Import libraries

In [1]:
from IPython.display import clear_output

In [2]:
!pip install fbgemm-gpu
#!pip install fbgemm-gpu-cpu

!pip install torch
!pip install torchrec
#!pip install torchrec-cpu
!pip install pyre_extensions

#clear_output()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.8/225.8 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.7/325.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 11.6 MB/s eta 0:00:00
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31530 sha256=f58d88d1f3ab56e5b436f49744fe3130d1bcb593a745ec1d3afe9b7b0aca80b4
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built iopath


In [3]:
import pandas as pd
import numpy as np
import random

import torch
from torch.utils.data.dataset import IterableDataset
from torchrec.datasets.utils import Batch
from torchrec.sparse.jagged_tensor import KeyedJaggedTensor

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

Device: 'cuda'


In [5]:
!git clone https://github.com/linafaik08/recommender_systems_dlrm.git

Cloning into 'recommender_systems_dlrm'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 59 (delta 13), reused 53 (delta 10), pack-reused 0
Receiving objects: 100% (59/59), 31.77 KiB | 10.59 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [6]:
cd recommender_systems_dlrm/src

/content/recommender_systems_dlrm/src


In [7]:
from batch import *
from model import *

## 1.2. Generate data

In [8]:
n_rows = 10000

train_df = pd.DataFrame({
    "feat1": np.random.rand(n_rows),
    "feat2": np.random.rand(n_rows),
    "cat1": np.random.choice(["A","B","C"], n_rows),
    "cat2": [random.choice([["A","B","C"], ["A","B"],  ["A","C"], ["B","C"], ["B"], []]) for i in range(n_rows)],
    "label": np.random.choice([0,1], n_rows)
})

n_rows = int(n_rows*0.2)

test_df = pd.DataFrame({
    "feat1": np.random.rand(n_rows),
    "feat2": np.random.rand(n_rows),
    "cat1": np.random.choice(["A","B","C"], n_rows),
    "cat2": [random.choice([["A","B","C"], ["A","B"],  ["A","C"], ["B","C"], ["B"], []]) for i in range(n_rows)],
    "label": np.random.choice([0,1], n_rows)
})

n_rows = int(n_rows*0.2)

val_df = pd.DataFrame({
    "feat1": np.random.rand(n_rows),
    "feat2": np.random.rand(n_rows),
    "cat1": np.random.choice(["A","B","C"], n_rows),
    "cat2": [random.choice([["A","B","C"], ["A","B"],  ["A","C"], ["B","C"], ["B"], []]) for i in range(n_rows)],
    "label": np.random.choice([0,1], n_rows)
})




In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# %cd ../..
%cd content
data_path = "drive/MyDrive/ReserveImportantDataCopy/MyProjects/Other/RSHBGrant/2023/Data/"

/content


In [23]:
import pandas as pd
# def regions ():
regions = pd.read_csv(data_path + "region.csv")
regions = regions[regions.columns[2]]
b = pd.Series(["Донецкая Народная Республика", "Луганская Народная Республика", "Запорожская область", "Херсонская область"], index=[86, 87, 88, 89])
regions = regions.append(b)
regions = regions.drop(85)
regions

<ipython-input-23-31cf9df708cb>:6: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  regions = regions.append(b)


0                       Респ Адыгея
1                 Респ Башкортостан
2                      Респ Бурятия
3                        Респ Алтай
4                     Респ Дагестан
                  ...              
84                    г Севастополь
86     Донецкая Народная Республика
87    Луганская Народная Республика
88              Запорожская область
89               Херсонская область
Length: 89, dtype: object

In [24]:
def generate_df(n_rows):
  df = pd.DataFrame({
  # step 0
  "houseArea": [random.randint(25, 250) for i in range(n_rows)],
  "siteArea": [random.randint(0, 100) for i in range(n_rows)],
  "floorCount": [random.randint(1, 3) for i in range(n_rows)],
  "region": [np.random.choice(regions) for i in range(n_rows)],
  "budgetFloor": [random.randint(500_000, 5_000_000) for i in range(n_rows)],
  "budgetCeil": [random.randint(5_000_000, 25_000_000) for i in range(n_rows)],

  # step 1
  # sitePreparation
  "siteChoosing": (np.random.rand(n_rows) >= 0.5),
  "geologicalWorks": np.random.rand(n_rows) >= 0.5,
  "geodeticalWorks": np.random.rand(n_rows) >= 0.5,
  "cuttingBushesAndSmallForests": np.random.rand(n_rows) >= 0.5,
  "clearingTheSiteOfDebris": np.random.rand(n_rows) >= 0.5,

  # siteWorks
  "cameras": np.random.rand(n_rows) >= 0.5,
  "temporaryFence": np.random.rand(n_rows) >= 0.5,

  # houseDesignAndProject
  "homeProject": np.random.rand(n_rows) >= 0.5,
  "designProject": np.random.rand(n_rows) >= 0.5,

  # step 2
  "foundationType": [np.random.choice(["Свайный", "Столбчатый", "Ленточный", "Плитный"]) for i in range(n_rows)],

  # step 3
  "wallsMaterial":[np.random.choice(["Кирпич", "Легкий бетон", "Дерево", "Каркас"]) for i in range(n_rows)],

  # step 4
  "slopesNumber": [random.randint(1, 4) for i in range(n_rows)],
  "roofType": [np.random.choice(["Металлочерепица", "Гибкая черепица", "Рулонные материалы", "Ондулин", "Профнастил"]) for i in range(n_rows)],

  # step 5
  "facadeTechnology": [np.random.choice(["Без отделки", "Панели (сайдинг)", "Облицовка кирпичом", "Искусственный камень"]) for i in range(n_rows)],

  # step 6
  "windowMaterial": [np.random.choice(["Деревянные", "Пластиковые"]) for i in range(n_rows)],
  "windowType": [np.random.choice(["Однокамерные", "Двухкамерные", "Трехкамерные"]) for i in range(n_rows)],
  "doorMaterial": [np.random.choice(["Деревянные", "Пластиковые"]) for i in range(n_rows)],

  # step 7
  # electrician
  "plasticBoxesUpTo40mmWide": np.random.rand(n_rows) >= 0.5,
  "layingAThreeToFive": np.random.rand(n_rows) >= 0.5,
  "cableLaying": np.random.rand(n_rows) >= 0.5,
  "installationOfTwoKey": np.random.rand(n_rows) >= 0.5,
  "installationOfSingleKey": np.random.rand(n_rows) >= 0.5,
  "recessedTypeSocketDevice": np.random.rand(n_rows) >= 0.5,
  "installationOfPendant": np.random.rand(n_rows) >= 0.5,
  "chandeliersAndPendants": np.random.rand(n_rows) >= 0.5,

  # waterSupply
  "layingOfInternalPipelines": np.random.rand(n_rows) >= 0.5,
  "installationOfBathtubs": np.random.rand(n_rows) >= 0.5,
  "installationOfSingle": np.random.rand(n_rows) >= 0.5,
  "installationOfMixer": np.random.rand(n_rows) >= 0.5,

  # sewerage
  "installationOfToilet": np.random.rand(n_rows) >= 0.5,
  "layingOfSewerage50mm": np.random.rand(n_rows) >= 0.5,
  "layingOfSewerage110mm": np.random.rand(n_rows) >= 0.5,

  # heating
  "assemblyOfAWaterSupply": np.random.rand(n_rows) >= 0.5,
  "layingOfInternalPipelines": np.random.rand(n_rows) >= 0.5,
  "installationOfWindowFixtures": np.random.rand(n_rows) >= 0.5,

  # ventilation
  "installationOfSplitSystems": np.random.rand(n_rows) >= 0.5,
  "cablingOnABrickWall": np.random.rand(n_rows) >= 0.5,


  #step 8
  "warmFloor": np.random.rand(n_rows) >= 0.5,
  "ladderMaterial": [np.random.choice(["Дерево", "Бетон", "Металл"]) for i in range(n_rows)],

  #step 9
  "wallDecoration": [np.random.choice(["Декоративная штукатурка", "Покраска", "Обои", "Плитка"]) for i in range(n_rows)],
  "floorCovering": [np.random.choice(["Ламинат", "Линолеум"]) for i in range(n_rows)],
  "ceilCovering": [np.random.choice(["Натяжной потолок", "Окраска", "Обои", "Штукатурка"]) for i in range(n_rows)],

  })
  return df
train_df = generate_df(n_rows)
test_df = generate_df(int(n_rows * 0.2))
val_df = generate_df(int(n_rows * 0.2 * 0.2))

In [25]:
train_df.head()

,houseArea,siteArea,floorCount,region,budgetFloor,budgetCeil,siteChoosing,geologicalWorks,geodeticalWorks,cuttingBushesAndSmallForests,...,layingOfSewerage110mm,assemblyOfAWaterSupply,installationOfWindowFixtures,installationOfSplitSystems,cablingOnABrickWall,warmFloor,ladderMaterial,wallDecoration,floorCovering,ceilCovering
0,136,1,2,Респ Крым,2226651,12259193,False,False,True,False,...,True,True,False,True,True,False,Дерево,Плитка,Ламинат,Обои
1,241,55,1,Респ Калмыкия,1714271,13303054,False,False,True,False,...,True,False,True,True,True,False,Бетон,Покраска,Линолеум,Штукатурка
2,127,64,3,Архангельская обл,2803340,9411575,True,True,False,True,...,False,False,True,True,True,True,Бетон,Обои,Линолеум,Окраска
3,189,29,3,Пермский край,795860,19923257,False,True,True,True,...,True,True,False,False,True,False,Дерево,Плитка,Линолеум,Окраска
4,124,51,3,Тульская обл,1366513,23611893,True,False,False,True,...,False,True,False,False,True,False,Дерево,Обои,Ламинат,Штукатурка


In [26]:
train_df.columns

Index(['houseArea', 'siteArea', 'floorCount', 'region', 'budgetFloor',
       'budgetCeil', 'siteChoosing', 'geologicalWorks', 'geodeticalWorks',
       'cuttingBushesAndSmallForests', 'clearingTheSiteOfDebris', 'cameras',
       'temporaryFence', 'homeProject', 'designProject', 'foundationType',
       'wallsMaterial', 'slopesNumber', 'roofType', 'facadeTechnology',
       'windowMaterial', 'windowType', 'doorMaterial',
       'plasticBoxesUpTo40mmWide', 'layingAThreeToFive', 'cableLaying',
       'installationOfTwoKey', 'installationOfSingleKey',
       'recessedTypeSocketDevice', 'installationOfPendant',
       'chandeliersAndPendants', 'layingOfInternalPipelines',
       'installationOfBathtubs', 'installationOfSingle', 'installationOfMixer',
       'installationOfToilet', 'layingOfSewerage50mm', 'layingOfSewerage110mm',
       'assemblyOfAWaterSupply', 'installationOfWindowFixtures',
       'installationOfSplitSystems', 'cablingOnABrickWall', 'warmFloor',
       'ladderMaterial', 